## This script can be found on [GitHub](https://github.com/Basem-Qahtan/STEM-data-processing-scripts) along with other scripts for STEM data processing algorithms.

##### --------------------------------------------------------------------------------------------------------------------------------------------
# STEM-Cathodoluminescence datasets  

Cathodoluminescence (CL) data acquisition in TEM is used in applications such as LEDs to collect information on the wavelength of the emitted photons from quantum wells. For example, in some cases the sample is exposed to the electron beam for a short dwell time to avoid sample's movement. However, this will result in a low signal-to-noise ratio (SNR). This notebook increases the SNR of a noisy CL datasets through the principal component analysis (PCA) denoising.


## Author

* 15/12/2024 Basem Qahtan - Developed as part of a Ph.D. research project at the Italian Institute of Technology (IIT)/ University of Genoa (UniGe) in Italy and KAUST University in KSA.
## Changes

* 15/10/2024 Update: Added auto cropping and rebinning of EDX map prior to the principal component analysis (PCA) denoising.
* 10/11/2024 Update: removal of negative signals and high intense signals at random pixels (artifacts). 


## Requirements

* HyperSpy 1.7.4 or higher

## <a id='top'></a> Contents

1. <a href='#dat'> Specimen & Data</a>
2. <a href='#load'> Load the CL data</a>
3. <a href='#crop'> Crop negative values and cap the intensity of pixels with extreme intensity (if needed)</a>
4. <a href='#pca'> Apply PCA decomposition on the CL map </a>
5. <a href='#save'> Save the denoised CL dataset in hspy format</a>


# <a id='dat'></a> 1. Specimen & Data

Samples used in this example were test data as shown in figure 1 below with high noise (left)

<img src="images/CL data PCA denoising.png" style="height:400px;">
Figure 1: Reconstructed Cathodoluminescence (CL) data after PCA denoising resulting in an enhanced signal-to-noise ratio (SNR).


## Import required libraries

In [1]:
%matplotlib qt
import hyperspy.api as hs
import numpy as np
import scipy.misc
import scipy
import matplotlib.pyplot as plt
import os
import re


# <a id='load'></a> 2. Load the CL data


In [3]:
# Specify the directory path
directory = os.getcwd()

# Get a list of files in the directory
file_list = os.listdir(directory)

# Filter files with the .emd or .bcf extension, you may add your file extension here if it is different.
emd_files = [file for file in file_list if file.endswith(".emd") or file.endswith(".bcf") or file.endswith(".dm4")]

def extract_number(filename):
    # Extract the number from the filename using regex
    match = re.search(r'\((\d+)\)', filename)
    if match:
        return int(match.group(1))
    return 0  # Return 0 if no number is found

# Sort the list using the custom key function
sorted_emd_files = sorted(emd_files, key=extract_number)

intitial_k = []
dynamic_K = []

# Load and process the .emd files using Hyperspy
for file in sorted_emd_files:
    # Construct the full file path
    file_path = os.path.join(directory, file)
    
    # Load the file using Hyperspy
    SI = hs.load(file_path)  

    # Split the string based on the '.' delimiter
    name = file.split('.')[0]

    CL_found = False
    HAADF = None
    EDS = None
    CL = None

    # Check if SI is a list (subscriptable) or a single signal
    if isinstance(SI, list):
        for i, signal in enumerate(SI):
            if signal.metadata.General.title == 'HAADF':
                HAADF = signal
                HAADFindex = i
            elif signal.metadata.General.title in ("EDS", "EDX"):
                if signal.data.ndim == 3:  # Ensure it has three dimensions
                    EDS = signal
                    EDSindex = i
            elif signal.metadata.General.title == 'CLSpectrum' or signal.metadata.Signal.signal_type=="CL":
                CL = signal
                CL_found = True
    else:
        # SI is a single signal
        if SI.metadata.General.title == 'HAADF':
            HAADF = SI
        elif SI.metadata.General.title in ("EDS", "EDX"):
            if SI.data.ndim == 3:
                EDS = SI
        elif SI.metadata.General.title == 'CLSpectrum' or SI.metadata.Signal.signal_type=="CL":
            CL = SI
            CL_found = True

    if CL_found == False: 
        print("CL map is not found in the dataset ")

    # Print the extracted values
    print("Sample name:", name)
    print("HAADF:", "Found" if HAADF is not None else "Not found")
    print("EDX:", "Found" if EDS is not None else "Not found")
    print("CL map:", "Found" if CL is not None else "Not found in the dataset (Cant continue the processing of CL data using this script)")
    print("------------------------------------------------------")


Sample name: PCA_TestData_03_RedBlue_5kV_HighNoise
HAADF: Not found
EDX: Not found
CL map: Found
------------------------------------------------------


In [4]:
SI

<CLSpectrum, title: PCA_TestData_03_RedBlue_5kV_HighNoise, dimensions: (50, 50|167)>

In [5]:
# check the varibale CL contains the SI dataset
CL

<CLSpectrum, title: PCA_TestData_03_RedBlue_5kV_HighNoise, dimensions: (50, 50|167)>

#### When plotting the signal below: if you cant use the keyboard arrows to navigate through the signal or navigation axes, try to drag the small red square (shown in figure 2 below in the top left corner of the CL map) by holding the right click on the mouse 

In [6]:
CL.plot()

# <a id='crop'></a> 3. Crop negative values and cap the intensity of pixels with extreme intensity (if needed)

<img src="images/high intensity capping in CL data.png" style="height:400px;">
Figure 2: Cropping wavelengths counts with extremly high intensities (due to artifacts) as it will affect the components in PCA.


In [7]:
# Assuming you've already calculated max_intensities
max_intensities = np.max(CL.data, axis=2)

# Get the indices of the top 50 intensity values
top_30_indices = np.argpartition(max_intensities.flatten(), -30)[-30:]

# Get the actual top 50 intensity values
top_30_intensities = max_intensities.flatten()[top_30_indices]

# Sort the top 50 intensities in descending order
top_30_intensities_sorted = np.sort(top_30_intensities)[::-1]

In [8]:
# check the highest intensity counts in the top 30 pixels
top_30_intensities_sorted

array([14050.088  ,  2956.1416 ,   388.51495,   366.3483 ,   364.37494,
         342.3483 ,   334.0616 ,   321.13828,   319.94827,   318.76495,
         315.18164,   312.97836,   311.76495,   307.18158,   306.5616 ,
         305.68158,   305.30496,   300.93164,   295.59827,   294.43158,
         293.8483 ,   289.84824,   283.13828,   281.395  ,   281.3483 ,
         279.05496,   278.9316 ,   277.76495,   273.0983 ,   272.72165],
      dtype=float32)

In [9]:
# cap the max intensity across the CL map based on the mean value below:
top_30_intensities_sorted.mean()

854.2831

In [10]:
# Assuming 'cl_data' is your 3D CL dataset
cl_data_clipped = np.clip(CL, a_min=0, a_max=top_30_intensities_sorted.mean())#None   #top_50_intensities_sorted.mean()

In [11]:
#check the CL map after cropping negative / high intensity counts:
cl_data_clipped.plot()

# <a id='pca'></a> 4. Apply PCA decomposition on the CL map

<img src="images/PCA on CL data.png" style="height:450px;">
Figure 3: Cropping wavelengths counts with extremly high intensities (due to artifacts) as it will affect the components in PCA.


In [12]:
cl_data_clipped.change_dtype("float32")

cl_data_clipped.decomposition()

Decomposition info:
  normalize_poissonian_noise=False
  algorithm=SVD
  output_dimension=None
  centre=None


## Plot the scree plot and the individual components:

#### After plotting the decomposition results below: if you cant navigate between the components using the keyboard arrows, use the slider bar opened in 3rd smal window or in the notebook itself as shown in figure 4 below.

In [13]:
cl_data_clipped.plot_decomposition_results()
cl_data_clipped.plot_explained_variance_ratio()

<Axes: title={'center': 'PCA_TestData_03_RedBlue_5kV_HighNoise\nPCA Scree Plot'}, xlabel='Principal component index', ylabel='Proportion of variance'>

<img src="images/slider bar.png" style="height:450px;">
Figure 4: Alternative option to cycle beween the PCA components if keyboard arrows or bindings doesnt work

## Select the number of components to be used in reconstructing the CL map:

#### If you want to use the first 4 components you can type:
###### CLdata =cl_data_clipped.get_decomposition_model(4)
##### --------------------------------------------------------------------------------------------------
#### If you want to use the 1st, 2nd and 4th components you can type:
###### CLdata =cl_data_clipped.get_decomposition_model([0,1,3])   >>>> numbering starts from 0

In [14]:
CLdata=cl_data_clipped.get_decomposition_model(2)

In [15]:
CLdata

<CLSpectrum, title: PCA_TestData_03_RedBlue_5kV_HighNoise model from decomposition with 2 components, dimensions: (50, 50|167)>

In [16]:
CLdata.plot()

# <a id='save'></a> 5. Save the denoised CL dataset in hspy format


In [ ]:
CLdata.save(name+"  (PCA denoised).hspy")
